**Parte 1: Preprocesamiento de Datos**

In [ ]:
# Librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importamos el DataFrame.
path = "/content/drive/MyDrive/Skillnest/ML/CORES/dataset1.csv"
df = pd.read_csv(path)

In [ ]:
# Visualizamos nuestro DataFrame.
df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


Limpieza de Datos

In [ ]:
# Verificar valores nulos
df.isna().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [ ]:
# Porcentaje de nulos por columna
porcentaje_nulos = df.isna().mean().sort_values(ascending=False) * 100

# Mostrar en forma de tabla
porcentaje_nulos = porcentaje_nulos.round(2).reset_index()
porcentaje_nulos.columns = ["Columna", "Porcentaje de Nulos"]
porcentaje_nulos

,Columna,Porcentaje de Nulos
0,Age,0.0
1,Sex,0.0
2,ChestPainType,0.0
3,RestingBP,0.0
4,Cholesterol,0.0
5,FastingBS,0.0
6,RestingECG,0.0
7,MaxHR,0.0
8,ExerciseAngina,0.0
9,Oldpeak,0.0


In [ ]:
# Verificar filas duplicadas
df.duplicated().sum()

np.int64(0)

No existen valors nulos, ni duplicados, por ende, no se puede aplicar metodos como imputación, eliminación, etc.

In [ ]:
# Descripción de variables categóricas
df.describe(include="object")

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
count,918,918,918,918,918
unique,2,4,3,2,3
top,M,ASY,Normal,N,Flat
freq,725,496,552,547,460


Análisis de Variables Categóricas Sex (Sexo) Valores únicos: 2 (M y F). Valor más frecuente: M (masculino), con 725 registros. Aproximadamente 79% de los pacientes son hombres, lo que indica un desbalance de género en la muestra. Esto es importante porque el riesgo de enfermedad cardíaca puede diferir entre sexos.

ChestPainType (Tipo de Dolor en el Pecho) Valores únicos: 4 (ASY, NAP, ATA, TA). Valor más frecuente: ASY (Asintomático), con 496 casos.

RestingECG (Resultado del ECG en reposo) Valores únicos: 3 (Normal, ST, LVH). Valor más frecuente: Normal, con 552 registros.
ExerciseAngina (Angina inducida por ejercicio) Valores únicos: 2 (Y, N). Valor más frecuente: N (sin angina), con 547 registros.
ST_Slope (Pendiente del segmento ST en ECG) Valores únicos: 3 (Up, Flat, Down). Valor más frecuente: Flat, con 460 casos

In [ ]:
# Descripción de variables numéricas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Age,918.0,53.51,9.43,28.0,47.00,54.0,60.0,77.0
RestingBP,918.0,132.40,18.51,0.0,120.00,130.0,140.0,200.0
Cholesterol,918.0,198.80,109.38,0.0,173.25,223.0,267.0,603.0
FastingBS,918.0,0.23,0.42,0.0,0.00,0.0,0.0,1.0
MaxHR,918.0,136.81,25.46,60.0,120.00,138.0,156.0,202.0
Oldpeak,918.0,0.89,1.07,-2.6,0.00,0.6,1.5,6.2
HeartDisease,918.0,0.55,0.50,0.0,0.00,1.0,1.0,1.0


Análisis Estadístico de Variables Numéricas Age (Edad) Media: 53.5 años, mediana: 54, lo que indica una distribución bastante simétrica. Rango: de 28 a 77 años. La mayoría de los pacientes tienen entre 47 y 60 años. Se trata de una población mayoritariamente adulta o adulta mayor, lo cual es esperable en estudios de enfermedades cardíacas.

**RestingBP** Media: 132 mmHg, ligeramente por encima del valor normal (menor a 120). Mínimo: 0, lo cual no es un valor fisiológicamente válido, posible error de registro. Rango típico: entre 120 y 140 mmHg. Aunque la mayoría tiene presión entre lo esperado, el valor mínimo sugiere un posible dato anómalo o faltante introducido como 0.

**Cholesterol** Media: 198.8 mg/dL (valor dentro de lo normal), pero con una alta desviación estándar de 109.4. Mínimo: 0 mg/dL también inválido fisiológicamente, otro caso de posible error o valor faltante. Percentiles 25 a 75: entre 173 y 267, lo que indica que muchos pacientes tienen colesterol elevado. Hay posibles registros inválidos (colesterol 0) y una alta variabilidad, lo que puede influir mucho en modelos predictivos si no se trata.

FastingBS Variable binaria (0 = no, 1 = sí). Promedio: 0.23 indica que solo ~23%.

MaxHR Media: 136.8 lpm. Rango: 60 a 202. Percentiles 25 a 75: entre 120 y 156, bastante disperso.

**Oldpeak** Media: 0.89, pero con valores tan bajos como -2.6 Máximo: 6.2. IQR: 0.0 a 1.5 la mayoría tiene valores bajos, pero hay valores extremos.

HeartDisease (Enfermedad cardíaca: 0 = no, 1 = sí) Media: 0.55 significa que un 55% de los pacientes tiene enfermedad cardíaca. Mediana = 1, lo que confirma que hay ligero desbalance hacia la clase positiva (enfermos).

**Análisis outliers**

En base al análisis estadístico realizado previamente sobre las variables numéricas, se detectaron algunos valores anómalos, como presiones arteriales y niveles de colesterol igual a 0, los cuales no son fisiológicamente válidos.

A continuación, se utilizarán boxplots para visualizar posibles outliers de forma gráfica. Esta exploración permitirá determinar si es necesario aplicar técnicas de filtrado o imputación como parte del preprocesamiento de datos.

Se eligió la variable HeartDisease como eje de comparación en los boxplots debido a que representa el diagnóstico clínico central del dataset (0 = sin enfermedad, 1 = con enfermedad).

Esta decisión permite analizar cómo varían las variables clínicas como presión arterial (RestingBP), colesterol (Cholesterol) y Oldpeak en función de si el paciente fue diagnosticado con una enfermedad cardíaca o no.

Esta comparación es clave porque nos ayuda a detectar patrones o diferencias significativas entre pacientes enfermos y sanos, lo cual es especialmente útil si el objetivo del proyecto es desarrollar un modelo predictivo de clasificación binaria para detectar enfermedad cardíaca.

In [ ]:
# Boxplot de Presión Arterial (RestingBP) según presencia de enfermedad cardíaca
fig = px.box(
    df,
    x="HeartDisease",
    y="RestingBP",
    color="HeartDisease",
    title="Distribución de Presión Arterial según Presencia de Enfermedad Cardíaca",
    points="all"
)
fig.update_layout(title_font_size=18)
fig.show()


La presión arterial en reposo (RestingBP) presenta una distribución similar en ambos grupos (con y sin enfermedad cardíaca), pero se observan valores atípicos hacia los extremos en ambos casos.
Sin embargo, el grupo con enfermedad cardíaca tiende a tener una mediana ligeramente mayor, lo cual podría indicar una mayor presión en reposo entre pacientes enfermos.
También se identifican valores de presión arterial igual a 0, que son fisiológicamente inválidos y deben ser tratados como valores anómalos o posiblemente faltantes codificados incorrectamente.

In [ ]:
# Boxplot de Colesterol (Cholesterol) según presencia de enfermedad cardíaca
fig = px.box(
    df,
    x="HeartDisease",
    y="Cholesterol",
    color="HeartDisease",
    title="Distribución de Colesterol según Presencia de Enfermedad Cardíaca",
    points="all"
)
fig.update_layout(title_font_size=18)
fig.show()

El nivel de colesterol presenta una mayor dispersión en el grupo de pacientes con enfermedad cardíaca.
La mediana es ligeramente mayor en ese grupo, y se observan valores extremos altos, lo cual es clínicamente consistente con el riesgo cardiovascular.
Al igual que con RestingBP, se detectan valores de colesterol igual a 0, lo que representa un error o valor faltante que deberá ser tratado en el preprocesamiento.

In [ ]:
# Boxplot de Oldpeak según presencia de enfermedad cardíaca
fig = px.box(
    df,
    x="HeartDisease",
    y="Oldpeak",
    color="HeartDisease",
    title="Distribución de Oldpeak según Presencia de Enfermedad Cardíaca",
    points="all"
)
fig.update_layout(title_font_size=18)
fig.show()

Oldpeak muestra una clara diferencia entre los grupos: los pacientes con enfermedad cardíaca tienden a tener valores más altos de Oldpeak, con una mediana visiblemente mayor.
También hay una mayor concentración de valores atípicos en ese grupo, lo cual puede reflejar una mayor depresión del segmento ST.
Esta variable parece ser discriminativa entre los dos grupos y podría ser relevante en modelos predictivos.

A modo de resumen:

**RestingBP**:	Mediana ligeramente mayor en pacientes con enfermedad. Se observan varios valores extremos.	Sí, incluyendo valores de 0 que no son válidos fisiológicamente.

**Cholesterol**:	Mayor dispersión y mediana más alta en pacientes con enfermedad cardíaca.	Sí, con valores extremos altos. También se observan valores 0 inválidos.

En la revisión de las variables numéricas RestingBP (presión arterial en reposo) y Cholesterol (colesterol), se identificaron valores iguales a 0,una presión arterial o un nivel de colesterol igual a 0 indica un error de registro, ya que fisiológicamente esos valores no pueden existir, por ende, seran tratados como NaN.
Posteriormente, se reemplazan con la mediana de cada variable.

**Oldpeak**:	Diferencia clara entre grupos; pacientes enfermos con valores más altos.	Sí, especialmente en pacientes con enfermedad.
Se identificaron múltiples registros con valores iguales a cero y valores negativos.
Se tomaron las siguientes decisiones:
Los valores igual a 0.0 se mantienen, ya que representan una condición clínicamente válida, ya que solo representa ausencia.

Los valores negativos fueron reemplazados por 0.0, ya que dichos valores no son posibles en el contexto clínico: no puede haber un valor negativo.

In [ ]:
import numpy as np

# Reemplazamos los valores 0 por np.nan en columnas específicas
df["RestingBP"] = df["RestingBP"].replace(0, np.nan)
df["Cholesterol"] = df["Cholesterol"].replace(0, np.nan)

In [ ]:
#Verificar cuántos nulos quedaron
df[["RestingBP", "Cholesterol"]].isna().sum()

,0
RestingBP,1
Cholesterol,172


In [ ]:
# Imputar con la mediana
df["RestingBP"].fillna(df["RestingBP"].median(), inplace=True)
df["Cholesterol"].fillna(df["Cholesterol"].median(), inplace=True)

/tmp/ipython-input-16-3462645527.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipython-input-16-3462645527.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)

In [ ]:
# Confirmar que ya no hay nulos
df[["RestingBP", "Cholesterol"]].isna().sum()

,0
RestingBP,0
Cholesterol,0


In [ ]:
# Filtrar registros con valores negativos en Oldpeak
oldpeak_negativos = df[df["Oldpeak"] < 0]

# Mostrar cuántos hay
print(f"Cantidad de registros con Oldpeak negativo: {oldpeak_negativos.shape[0]}")

# Mostrar los registros con Oldpeak < 0
oldpeak_negativos

Cantidad de registros con Oldpeak negativo: 13


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
321,63,M,ASY,100.0,237.0,1,Normal,109,N,-0.9,Flat,1
324,46,M,ASY,100.0,237.0,1,ST,133,N,-2.6,Flat,1
325,42,M,ASY,105.0,237.0,1,Normal,128,Y,-1.5,Down,1
326,45,M,NAP,110.0,237.0,0,Normal,138,N,-0.1,Up,0
331,56,M,ASY,115.0,237.0,1,ST,82,N,-1.0,Up,1
332,38,M,NAP,100.0,237.0,0,Normal,179,N,-1.1,Up,0
334,51,M,ASY,130.0,237.0,1,Normal,170,N,-0.7,Up,1
335,62,M,TA,120.0,237.0,1,LVH,134,N,-0.8,Flat,1
352,56,M,ASY,120.0,237.0,0,ST,100,Y,-1.0,Down,1
407,62,M,ASY,115.0,237.0,1,Normal,72,Y,-0.5,Flat,1


In [ ]:
# Reemplazar valores negativos de Oldpeak por 0.0
df.loc[df["Oldpeak"] < 0, "Oldpeak"] = 0.0

In [ ]:
# Verificar
(df["Oldpeak"] < 0).sum()

np.int64(0)

In [ ]:
# Revisando de forma general
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Age,918.0,53.51,9.43,28.0,47.0,54.0,60.0,77.0
RestingBP,918.0,132.54,17.99,80.0,120.0,130.0,140.0,200.0
Cholesterol,918.0,243.20,53.40,85.0,214.0,237.0,267.0,603.0
FastingBS,918.0,0.23,0.42,0.0,0.0,0.0,0.0,1.0
MaxHR,918.0,136.81,25.46,60.0,120.0,138.0,156.0,202.0
Oldpeak,918.0,0.90,1.05,0.0,0.0,0.6,1.5,6.2
HeartDisease,918.0,0.55,0.50,0.0,0.0,1.0,1.0,1.0


Variables:

Variables numericas: "Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak".

Age (Edad):
Es una variable numérica continua que representa la edad en años. Tiene un orden natural.

RestingBP (Presión arterial en reposo):
Variable numérica continua que mide la presión arterial sistólica en mmHg.

Cholesterol (Colesterol):
Variable numérica continua que representa la concentración de colesterol en sangre, medida en mg/dL.

MaxHR (Frecuencia cardíaca máxima):
Variable numérica continua que representa la frecuencia cardíaca máxima alcanzada.

Oldpeak (Depresión ST):
Variable numérica continua.

Variables categoricas nominales: "Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope".

Sex (Sexo):
Categorías M (masculino) y F (femenino). No hay orden ni jerarquía entre estas categorías.

ChestPainType (Tipo de dolor en el pecho):
Categorías como ATA, NAP, ASY, etc. Son tipos diferentes de dolor, sin un orden inherente entre ellos.

RestingECG (ECG en reposo):
Categorías como Normal, ST, etc. Son distintos tipos de resultados, sin jerarquía o relación ordinal clara.

ExerciseAngina (Angina inducida por ejercicio):
Categorías Y (sí) y N (no). Son etiquetas categóricas sin orden.

ST_Slope (Pendiente del segmento ST):
Toma valores como "Up", "Flat", "Down".
Aunque podría pensarse que existe un orden clínico (ej. Up > Flat > Down), en la práctica las categorías representan tipos diferentes de pendientes, no "mejor" o "peor" necesariamente.

Variables categoricas ordinales: "FastingBS".

FastingBS (Glucosa en ayunas):
Solo toma dos valores: 0 o 1.
Estos valores sí representan un orden y un nivel de gravedad/condición:
0 = glucosa normal (condición "menor" o "mejor")
1 = glucosa alta (condición "mayor" o "peor")
Aquí hay un significado clínico claro que implica un orden natural: tener glucosa alta es peor que tenerla normal.
Por eso, aunque binaria, se considera ordinal, porque los valores implican una progresión o jerarquía.

Target: "HeartDisease".
HeartDisease toma valores 0 o 1:
0 = No tiene enfermedad cardíaca.
1 = Tiene enfermedad cardíaca.
Es la variable que quieres que el modelo prediga basándose en las otras características.



In [ ]:
# Definir features y target.
X = df.drop(columns=["HeartDisease"])
y = df["HeartDisease"]

In [ ]:
# Dividir en train y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir transformadores.
col_num = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]
col_ord=["FastingBS"]
col_nom = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

**Parte 2: Selección de Técnica de Machine Learning**

REGRESION LOGISTICA

In [ ]:
# Preprocesador rl
preprocessor_rl = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline rl
pipeline_rl= Pipeline(steps=[
    ("preprocessor", preprocessor_rl),
    ("classifier", LogisticRegression(max_iter=1000))
])

In [ ]:
# Entrenar modelo rl
pipeline_rl.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [ ]:
# Predecir rl
y_pred_rl = pipeline_rl.predict(X_test)

In [ ]:
# Evaluar rl
accuracy_rl = accuracy_score(y_test, y_pred_rl)

KNN

In [ ]:
# Preprocesador knn
preprocessor_knn = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline knn
pipeline_knn = Pipeline(steps=[
    ("preprocessor", preprocessor_knn),
    ("classifier", KNeighborsClassifier())
])

In [ ]:
# Entrenar modelo knn
pipeline_knn.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', KNeighborsClassifier())])

In [ ]:
# Predecir knn
y_pred_knn = pipeline_knn.predict(X_test)

In [ ]:
# Evaluar knn
accuracy_knn = accuracy_score(y_test, y_pred_knn)

ARBOL DE DECISION

In [ ]:
# Preprocesador tree
preprocessor_tree = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline tree
pipeline_tree = Pipeline(steps=[
    ("preprocessing", preprocessor_tree),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

In [ ]:
# Entrenar modelo tree
pipeline_tree.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [ ]:
# Predecir tree
y_pred_tree = pipeline_tree.predict(X_test)

In [ ]:
# Evaluar tree
accuracy_tree = accuracy_score(y_test, y_pred_tree)

RANDOM FOREST

In [ ]:
# Preprocesador rf
preprocessor_rf = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline rf
pipeline_rf = Pipeline(steps=[
    ("preprocessor", preprocessor_rf),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [ ]:
# Entrenar modelo rf
pipeline_rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
# Predecir rf
y_pred_rf = pipeline_rf.predict(X_test)

In [ ]:
# Evaluar rf
accuracy_rf = accuracy_score(y_test, y_pred_rf)

XGBoost

In [ ]:
# Preprocesador xgb
preprocessor_xgb = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline xgb
pipeline_xgb = Pipeline(steps=[
    ("preprocessor", preprocessor_xgb),
    ("classifier", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss"))
])

In [ ]:
# Entrenar modelo xgb
pipeline_xgb.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:28:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=Non...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [ ]:
# Predecir xgb
y_pred_xgb = pipeline_xgb.predict(X_test)

In [ ]:
# Evaluar xgb
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

LGBM

In [ ]:
# Preprocesador lgbm
preprocessor_lgbm = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline lgbm
pipeline_lgbm = Pipeline(steps=[
    ("preprocessor", preprocessor_lgbm),
    ("classifier", LGBMClassifier(random_state=42))
])

In [ ]:
# Entrenar modelo lgbm
pipeline_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 401, number of negative: 333
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 734, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546322 -> initscore=0.185819
[LightGBM] [Info] Start training from score 0.185819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', LGBMClassifier(random_state=42))])

In [ ]:
# Predecir lgbm
y_pred_lgbm = pipeline_lgbm.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [ ]:
# Evaluar lgbm
accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)

Comparación de Modelos

In [ ]:
# Resultados
print(f"Accuracy del modelo RL: {accuracy_rl:.4f}")
print(f"Accuracy del modelo KNN: {accuracy_knn:.4f}")
print(f"Accuracy del modelo TREE: {accuracy_tree:.4f}")
print(f"Accuracy del modelo RF: {accuracy_rf:.4f}")
print(f"Accuracy del modelo XGBOOST: {accuracy_xgb:.4f}")
print(f"Accuracy del modelo LGBM: {accuracy_lgbm:.4f}")

Accuracy del modelo RL: 0.8587
Accuracy del modelo KNN: 0.8533
Accuracy del modelo TREE: 0.7989
Accuracy del modelo RF: 0.8641
Accuracy del modelo XGBOOST: 0.8587
Accuracy del modelo LGBM: 0.8587


Se evaluaron seis modelos de clasificación para el problema de predicción binaria, obteniendo los siguientes resultados:

Accuracy del modelo RL: 0.8587
Accuracy del modelo KNN: 0.8533
Accuracy del modelo TREE: 0.7989
Accuracy del modelo RF: 0.8641
Accuracy del modelo XGBOOST: 0.8587
Accuracy del modelo LGBM: 0.8587

El modelo Random Forest (RF) obtuvo el mayor accuracy de 0.8641, superando a los demás modelos.

**Parte 3: Optimización de Hiperparámetros**

GridSearchCV

In [ ]:
# Preprocesador completo
preprocessor_forest = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
# Pipeline RandomForestClassifier
pipeline_forest = Pipeline([
    ("pp", preprocessor_forest),
    ("model", RandomForestClassifier(random_state=42))
])

In [ ]:
forest_params = {
    "model__n_estimators": [50, 100, 200]
}

In [ ]:
# GridSearchCV con 5 folds y accuracy como métrica
forest_grid = GridSearchCV(pipeline_forest, forest_params, cv=3, scoring="accuracy", n_jobs=-1)
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pp',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'RestingBP',
                                                                          'Cholesterol',
                                                                          'MaxHR',
                                                                          'Oldpeak']),
                                                                        ('ord',
                                                                         OrdinalEncoder(),
                                                                         ['FastingBS']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Sex',
                                                                          'ChestPainType',
                                                                          'RestingECG',
                                                                          'ExerciseAngina',
                                                                          'ST_Slope'])])),
                                       ('model',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1, param_grid={'model__n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [ ]:
# Mejor modelo y evaluación en test
forest_best = forest_grid.best_estimator_
y_pred_forest = forest_best.predict(X_test)

Optuna

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

In [ ]:
# Preprocesador completo
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("ord", OrdinalEncoder(), col_ord),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [ ]:
def objective_clf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
    }

    model = Pipeline([
        ("pre", preprocessor),
        ("rf", RandomForestClassifier(random_state=42, **params))
    ])

    score = cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1)
    return score.mean()

study_clf = optuna.create_study(direction="maximize")
study_clf.optimize(objective_clf, n_trials=20)

optuna_bparams = study_clf.best_params

print("Mejores hiperparámetros (clasificación):", optuna_bparams)

[I 2025-07-10 20:43:47,074] A new study created in memory with name: no-name-eba61d60-d69b-476f-bf82-cc3adb4d9951
[I 2025-07-10 20:43:50,825] Trial 0 finished with value: 0.8678041708486672 and parameters: {'n_estimators': 165, 'max_depth': 24}. Best is trial 0 with value: 0.8678041708486672.
[I 2025-07-10 20:43:51,173] Trial 1 finished with value: 0.8650830824132932 and parameters: {'n_estimators': 58, 'max_depth': 22}. Best is trial 0 with value: 0.8678041708486672.
[I 2025-07-10 20:43:52,088] Trial 2 finished with value: 0.8678153228504516 and parameters: {'n_estimators': 170, 'max_depth': 27}. Best is trial 2 with value: 0.8678153228504516.
[I 2025-07-10 20:43:54,068] Trial 3 finished with value: 0.8678153228504516 and parameters: {'n_estimators': 415, 'max_depth': 6}. Best is trial 2 with value: 0.8678153228504516.
[I 2025-07-10 20:43:54,602] Trial 4 finished with value: 0.8650830824132932 and parameters: {'n_estimators': 61, 'max_depth': 26}. Best is trial 2 with value: 0.8678153

Mejores hiperparámetros (clasificación): {'n_estimators': 97, 'max_depth': 14}


In [ ]:
# Pipeline final con los mejores hiperparámetros encontrados por Optuna
model = Pipeline([
    ("pp", preprocessor_forest),
    ("model", RandomForestClassifier(random_state=42, **optuna_bparams))
])


In [ ]:
# Entrenamiento.
model.fit(X_train, y_train)

Pipeline(steps=[('pp',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak']),
                                                 ('ord', OrdinalEncoder(),
                                                  ['FastingBS']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('model',
                 RandomForestClassifier(max_depth=14, n_estimators=97,
                                        random_state=42))])

In [ ]:
# Prediccion.
y_pred = model.predict(X_test)

Evaluación de Modelos Optimizados

In [ ]:
# Evaluacion
print("Random Forest Classifier")
print("Mejores parámetros:", forest_grid.best_params_)
print(f"Accuracy en test: {accuracy_score(y_test, y_pred_forest):.4f}")

print("Random Forest Classifier (Optuna)")
print("Mejores parámetros:", optuna_bparams)
print(f"Accuracy en test: {accuracy_score(y_test, y_pred):.4f}")

Random Forest Classifier
Mejores parámetros: {'model__n_estimators': 100}
Accuracy en test: 0.8641
Random Forest Classifier (Optuna)
Mejores parámetros: {'n_estimators': 97, 'max_depth': 14}
Accuracy en test: 0.8641


**CONCLUSION FINAL**
Tras identificar a Random Forest como el mejor modelo inicial con un accuracy de 0.8641, se procedió a optimizar sus hiperparámetros utilizando dos técnicas: GridSearchCV y Optuna.

La optimización con GridSearchCV encontró como mejor parámetro n_estimators = 100, manteniendo un accuracy de 0.8641 en el conjunto de prueba.

La optimización con Optuna sugirió parámetros similares (n_estimators = 97, max_depth = 14), logrando también un accuracy de 0.8641.

Estos resultados indican que el modelo Random Forest ya estaba cercano a su configuración óptima antes de la optimización. La mejora en accuracy no fue significativa, lo que sugiere que el modelo es estable y bien ajustado bajo las condiciones actuales del dataset y preprocesamiento.

Por lo tanto, la elección final del modelo Random Forest optimizado es adecuada para el problema planteado.

